In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score
from sklearn.linear_model import LinearRegression,Lasso,LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor
im

In [23]:
data = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/winequality-white.csv',sep=';')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [24]:
data.quality.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [25]:
X , y = data.drop('quality',axis=1) , data.quality
X.shape , y.shape

((4898, 11), (4898,))

In [26]:
X.isna().sum() 

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
dtype: int64

In [27]:
y.isna().sum()

0

In [28]:
X_train , X_valid , y_train , y_valid = train_test_split(X,y,test_size=0.3,random_state=17)
X_train.shape , X_valid.shape , y_train.shape , y_valid.shape

((3428, 11), (1470, 11), (3428,), (1470,))

In [30]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [32]:
%%time
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled,y_train)
print('Mean squared error : %f'%mean_squared_error(y_valid,lin_reg.predict(X_valid_scaled)))

Mean squared error : 0.584247
Wall time: 20.2 ms


In [47]:
pd.DataFrame({'Coef regression': lin_reg.coef_,
              'Coef regression abs':np.abs(lin_reg.coef_)}
             ,index = data.columns.drop('quality')).sort_values(by='Coef regression abs',ascending=False)

,Coef regression,Coef regression abs
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
fixed acidity,0.097822,0.097822
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127


In [50]:
%%time
lasso1 = Lasso(alpha=0.1,random_state=17)
lasso1.fit(X_train_scaled,y_train)
print('Mean squared error : %f'%mean_squared_error(y_valid,lasso1.predict(X_valid_scaled)))

Mean squared error : 0.617732
Wall time: 6.53 ms


In [51]:
pd.DataFrame({'Coef lasso':lasso1.coef_,
             'Coef lasso abs':np.abs(lasso1.coef_)},index=X_train.columns).sort_values(by='Coef lasso abs',ascending=False)

,Coef lasso,Coef lasso abs
alcohol,0.295922,0.295922
volatile acidity,-0.093247,0.093247
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000
residual sugar,0.000000,0.000000
chlorides,-0.000000,0.000000
free sulfur dioxide,0.000000,0.000000
total sulfur dioxide,-0.000000,0.000000
density,-0.000000,0.000000
pH,0.000000,0.000000


In [56]:
%%time
alphas = np.logspace(-6, 2, 200)
lasso2 =LassoCV(alphas=alphas,random_state=17,cv=5)
lasso2.fit(X_train_scaled,y_train)
print('Mean squared error : %f'%mean_squared_error(y_valid,lasso2.predict(X_valid_scaled)))

Mean squared error : 0.583298
Wall time: 330 ms


In [58]:
pd.DataFrame({'Coef lasso_cv':lasso2.coef_,
             'Coef lasso_cv abs':np.abs(lasso2.coef_)},index=X_train.columns).sort_values(by='Coef lasso_cv abs',ascending=False)

,Coef lasso_cv,Coef lasso_cv abs
density,-0.648161,0.648161
residual sugar,0.526883,0.526883
volatile acidity,-0.192049,0.192049
pH,0.146549,0.146549
alcohol,0.137115,0.137115
fixed acidity,0.093295,0.093295
sulphates,0.060939,0.060939
free sulfur dioxide,0.042698,0.042698
total sulfur dioxide,0.012969,0.012969
chlorides,0.006933,0.006933


In [72]:
%%time
bagging = BaggingRegressor(base_estimator=Lasso(alpha=0.0001),n_estimators=30,random_state=17,n_jobs=-1)
bagging.fit(X_train_scaled,y_train)
print('Mean squared error : %f'%mean_squared_error(y_valid,bagging.predict(X_valid_scaled)))

Mean squared error : 0.582315
Wall time: 320 ms


In [79]:
%%time
forest = RandomForestRegressor(random_state=17,n_jobs=-1)
forest.fit(X_train,y_train)
print('Mean squared error : %f'%mean_squared_error(y_valid,forest.predict(X_valid)))

Mean squared error : 0.370764
Wall time: 1.05 s


In [85]:
%%time
params = {
    'max_depth':range(1,32,5),
    'n_estimators':range(50,251,50),
    'max_features':range(1,20,2),
}
random_search_cv_forest = RandomizedSearchCV(estimator=RandomForestRegressor(),\
                                            param_distributions=params,random_state=17,n_jobs=-1,cv=10)
random_search_cv_forest.fit(X_train,y_train)
print('Mean squared error : %f'%mean_squared_error(y_valid,random_search_cv_forest.predict(X_valid)))

D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.50031165 0.49360298 0.45689969        nan        nan        nan
 0.36321413        nan        nan 0.49999818]
  warnings.warn(


Mean squared error : 0.372235
Wall time: 31.6 s


In [104]:
%%time
forest = RandomForestRegressor(max_depth=19,max_features=7,min_samples_leaf=1,random_state=17,n_jobs=-1)
forest.fit(X_train,y_train)
print('Mean squared score cross val : %f'%np.mean(np.abs(cross_val_score(estimator=forest,X=X_train,y=y_train,\
                                                                  cv=5,n_jobs=-1,scoring='neg_mean_squared_error'))))
print('Mean squared score : %f'%mean_squared_error(y_valid,forest.predict(X_valid)))

Mean squared score cross val : 0.403611
Mean squared score : 0.369435
Wall time: 3.16 s
